# Test a Tool in Isolation

The purpose of this notebook is to test a tool in isolation from the entire graph to confirm proper implementation.

**NOTE**: You must have your `GOOGLE_API_KEY` defined in your .env file. You can get this API key here: https://aistudio.google.com/app/apikey

In [2]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langchain_core.runnables import RunnableLambda
from langchain.tools import tool
%pip install --upgrade --quiet  langchain-community
import os
import pprint
from langchain_community.utilities import GoogleSerperAPIWrapper

os.environ["SERPER_API_KEY"] = "4369e38ae59aff075549b44c923813da127c06ef"

from currensee.core import get_model, settings

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv

load_dotenv()

True

## Import or Define Your Tool

Custom tools may be defined as follows:

```python
@tool
def add(a: float, b: float) -> float:
    """Add two float numbers"""
    return a + b
```

Tools may be imported as here:

```python
from langchain_community.agent_toolkits import O365Toolkit

toolkit = O365Toolkit()
o365_tools = toolkit.get_tools()

```

In [4]:
# Import/define your tool here

@tool
def add(a: float, b: float) -> float:
    """Add two float numbers"""
    return a + b

## Create Your Agent

An agent requires:
* the model
* tools
* a name
* a prompt describing the purpose of the tool

Example:



```python
math_agent = create_react_agent(
    model=model,
    tools=[add],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time.",
).with_config(tags=["skip_stream"])

```

In [5]:
# Define the gemini model for the tool
model = get_model(settings.DEFAULT_MODEL)

# Add your tools
tools = [add]

# Define the agent
name = "fin_market_expert"

# Define a prompt for the agent's purpose
prompt = "You are a financial market expert. Summarize these news into a few sentences."

In [6]:
test_agent = create_react_agent(
    model=model,
    tools=tools,
    name=name,
    prompt=prompt,
).with_config(tags=["skip_stream"])

## Prompt the agent

In [7]:
query = "Summarize these news into a few sentences."

In [8]:
inputs = {"input": query}
inputs = {"messages": [("human", query)]}

In [9]:
# Invoke agent
result = test_agent.invoke(input=inputs)

In [10]:
# Process output
print(result["messages"][-1])

content='What would you like to know about?  I need a topic to give you information on.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} name='fin_market_expert' id='run-62948a30-f6e2-482e-9dc7-13a76dd4e9a2-0' usage_metadata={'input_tokens': 31, 'output_tokens': 20, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}}


### FINANCIAL NEWS TOOLS

In [11]:
#initial synthesized prompt?

# Parameters
client_name = "Walmart"
start_date = "1/31/2025"
end_date = "4/30/2025"
industry = "retail"
#Future: portfolio tickers(for data and news)

#summary?

In [12]:
#definitions

keywords_client = ["announces", "acquires", "launches", "earnings", "report", 
               "profit", "CEO", "crisis", "disaster","recession","recovery", "red flag", 
               "urgent","challenge","emergency", "tumble","drop","opportunity","slowdown"]

keywords_econ = ["recovery","crisis", "disaster","recession","red flag", 
               "urgent","challenge","emergency", "tumble","drop","slowdown"]

trusted_sources = ["reuters.com", "bloomberg.com", "cnn.com", "forbes.com", 
                "finance.yahoo.com","marketwatch.com","morningstar.com", "https://www.wsj.com","www.ft.com"]

**FIN_TOOL_A - Search for top k news articles about the client and its industry (title, snippet, date, source)**

In [15]:

# Convert dates to YYYYMMDD format for Google's date range filter
def format_google_date(date_str):
    parts = date_str.split("/")
    return f"{parts[2]}{parts[0].zfill(2)}{parts[1].zfill(2)}"

google_start = format_google_date(start_date)
google_end = format_google_date(end_date)

# Trusted domains
allowed_sites = trusted_sources
site_filter = " OR ".join(f"site:{site}" for site in allowed_sites)

# Build query with date range filter
query = f"{site_filter} news about {client_name} and about {industry} industry"
sort_param = f"date:r:{google_start}:{google_end}"  # Google's date range format

# Search with date filter
search = GoogleSerperAPIWrapper(k=30, sort=sort_param)  # Pass sort parameter
results = search.results(query)

# Scoring logic (unchanged)
def score_result(result):
    score = 0
    keywords = keywords_client
    link = result.get("link", "")
    title = result.get("title", "").lower()
    snippet = result.get("snippet", "").lower()

    # Boost for trusted domain
    if any(site in link for site in allowed_sites):
        score += 3

    # Boost for keywords
    if any(word in title or word in snippet for word in keywords):
        score += 2

    # Boost for having a date
    if "date" in result:
        score += 1

    return score

# Process results
if results.get("organic"):
    sorted_results_client = sorted(results.get("organic", []), key=score_result, reverse=True)
    print(sorted_results_client)
    
    print(f"\nTop News about {client_name} from trusted sources ({start_date} → {end_date}):\n")
    for i, result in enumerate(sorted_results, 1):
        title = result.get("title")
        link = result.get("link")
        snippet = result.get("snippet")
        date = result.get("date", "Date not available")
        print(f"{i}. {title} ({date})\n   {snippet}\n   🔗 {link}\n")
else:
    print("\nNo results found for the query.")

[{'title': "Walmart's Online Business on Track to Be Profitable This Year", 'link': 'https://www.bloomberg.com/news/articles/2025-04-09/walmart-s-online-business-on-track-to-be-profitable-this-year', 'snippet': "Walmart Inc.'s US online business is on track to reach profitability this year, Chief Financial Officer John David Rainey said Wednesday.", 'date': 'Apr 9, 2025', 'position': 3}, {'title': 'Walmart, Target, Nike stocks drop as sweeping reciprocal tariffs are ...', 'link': 'https://finance.yahoo.com/news/walmart-target-nike-stocks-drop-as-sweeping-reciprocal-tariffs-are-expected-to-hike-consumer-prices-213339162.html', 'snippet': 'What has been confirmed today is that there is no escape from tariffs. Every company that imports is going to have to deal with higher costs ...', 'date': 'Apr 3, 2025', 'position': 11}, {'title': "Walmart's Tepid Outlook Stirs Concern For Retail Earnings Ahead", 'link': 'https://www.bloomberg.com/news/articles/2025-02-22/walmart-s-tepid-outlook-stirs-

**FIN_TOOL_B - Search for top k news articles about the client's holdings (title, snippet, date, source)**

**FIN_TOOL_C - Search for relevant articles about the economy (title, snippet, date, source)**

In [16]:

# Convert dates to YYYYMMDD format for Google's date range filter
def format_google_date(date_str):
    parts = date_str.split("/")
    return f"{parts[2]}{parts[0].zfill(2)}{parts[1].zfill(2)}"

google_start = format_google_date(start_date)
google_end = format_google_date(end_date)

# Trusted domains
allowed_sites = trusted_sources
site_filter = " OR ".join(f"site:{site}" for site in allowed_sites)

# Build query with date range filter
query = f"{site_filter} news about relevant macro events and the economy."
sort_param = f"date:r:{google_start}:{google_end}"  # Google's date range format

# Search with date filter
search = GoogleSerperAPIWrapper(k=30, sort=sort_param)  # Pass sort parameter
results = search.results(query)

# Scoring logic (unchanged)
def score_result(result):
    score = 0
    keywords = keywords_econ
    link = result.get("link", "")
    title = result.get("title", "").lower()
    snippet = result.get("snippet", "").lower()

    # Boost for trusted domain
    if any(site in link for site in allowed_sites):
        score += 3

    # Boost for keywords
    if any(word in title or word in snippet for word in keywords):
        score += 2

    # Boost for having a date
    if "date" in result:
        score += 1

    return score

# Process results
if results.get("organic"):
    sorted_results_econ = sorted(results.get("organic", []), key=score_result, reverse=True)
    print(sorted_results_econ)
    
    print(f"\nTop News about {client_name} from trusted sources ({start_date} → {end_date}):\n")
    for i, result in enumerate(sorted_results, 1):
        title = result.get("title")
        link = result.get("link")
        snippet = result.get("snippet")
        date = result.get("date", "Date not available")
        print(f"{i}. {title} ({date})\n   {snippet}\n   🔗 {link}\n")
else:
    print("\nNo results found for the query.")

[{'title': "US economy 'dangerously vulnerable' to recession - Yahoo Finance", 'link': 'https://finance.yahoo.com/news/us-economy-dangerously-vulnerable-to-recession-wall-street-reacts-to-tariff-fallout-155837444.html', 'snippet': 'President Trump laid out his long-awaited tariff plans — and Wall Street economists are reacting with recession warnings.', 'date': 'Apr 3, 2025', 'position': 16}, {'title': "Whisper it and it's back: Recession risk creeps onto markets' radar", 'link': 'https://www.reuters.com/markets/us/global-markets-growth-analysis-2025-03-05/', 'snippet': 'Global growth concerns have shot back onto the radar of financial markets as weakening U.S. economic data and growing trade tensions hurt ...', 'date': 'Mar 4, 2025', 'attributes': {'Missing': 'relevant | Show results with:relevant'}, 'position': 20}, {'title': "America's economic engine is still revving, despite fears of a slowdown", 'link': 'https://www.cnn.com/2024/09/17/economy/us-retail-sales-august/index.html', '

In [ ]:
**FIN_TOOL_D - Provide a generate market update on the latest macro/financial indicators**

In [ ]:
**FIN_TOOL_E - Provide a generate market update on the latest macro/financial indicators**

In [150]:
from transformers import pipeline

# Load summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
all_snippets = " ".join(result.get("snippet", "") for result in sorted_results)

# Chunking logic (BART has a 1024-token limit)
def chunk_text(text, max_tokens=1000):
    words = text.split()
    for i in range(0, len(words), max_tokens):
        yield " ".join(words[i:i+max_tokens])

chunks = list(chunk_text(all_snippets))
summaries = [summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text'] for chunk in chunks]

# Combine all chunk summaries into one final summary
final_summary = summarizer(" ".join(summaries), max_length=350, min_length=150, do_sample=False)[0]['summary_text']

print(f"\n📝 Summary of All News About {client_name} ({start_date} → {end_date}):\n{final_summary}\n")

Your max_length is set to 350, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)



📝 Summary of All News About Walmart (1/31/2025 → 4/30/2025):
More than half of all Walmart U.S. store managers earned bonuses of $100,000 or more. The company's proxy statement revealed its executive pay and laid out issues set to come up at its annual meeting. Walmart (WMT) closed at $92.41 in the latest trading session, marking a -0.87% move from the prior day. The stock is down 0.9% this year. The Dow Jones Industrial Average is up 0.7%. The S&P 500 is up 1.2%. The Nasdaq is up 2.3%. The Russell 2000 is up 3.1%. The NASDAQ is down 1.4%. The FTSE 100 is down 2.1% this week.



In [155]:
#!pip install sentencepiece
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("t5-small")  # <- safer for weird installs
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Combine all snippets into one block
all_snippets = " ".join(result.get("snippet", "") for result in sorted_results)
input_text = "summarize: " + all_snippets.strip()

# T5 input is limited to 512 tokens for t5-small
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(inputs, max_length=100, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(f"\n📝 Summary of All News About {client_name} ({start_date} → {end_date}):\n{summary}\n")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`



📝 Summary of All News About Walmart (1/31/2025 → 4/30/2025):
more than half of all Walmart U.S. store managers earned bonuses of $100,000 or more. analysts say that Walmart can use its enormous scale and wide supplier base to keep prices down for customers, even as tariffs raise its costs. president Donald Trump met with major retailers at the white house.



In [ ]:
# #USING LLM
# #!pip install newsapi-python
# #!pip install scikit-learn
# from google.cloud import aiplatform
# from newsapi import NewsApiClient
# from datetime import datetime, timedelta
# from sklearn.metrics.pairwise import cosine_similarity

# # Set up Google Cloud and News API
# newsapi = NewsApiClient(api_key='b42a6a3dbdd24443965c617126242eaa')
# aiplatform.init(project="adsp-34002-on02-sopho-scribe", location="us-central1")

# query = 'Walmart AND (earnings OR stock OR supply chain OR market OR revenue OR financial OR sales OR investment)'

# def fetch_news(query="Walmart", days_back=2):
#     date_from = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')

#     smart_query = 'Walmart AND (earnings OR stock OR supply chain OR market OR revenue OR financial OR sales OR investment)'
    
#     articles = newsapi.get_everything(
#         q=smart_query,
#         from_param=date_from,
#         language='en',
#         sort_by='relevancy',
#         page_size=10,
#         domains='reuters.com,bloomberg.com,wsj.com,cnn.com,forbes.com'
#     )

#     return [
#         a.get('title', 'No title') + ": " + a.get('description', 'No description available')
#         for a in articles.get('articles', [])
#         if a.get('description') and len(a.get('description')) > 30
#     ]

# def summarize_with_gemini(texts):
#     endpoint = "projects/adsp-34002-on02-sopho-scribe/locations/us-central1/endpoints/YOUR_ENDPOINT_ID"
#     model = aiplatform.gapic.PredictionServiceClient()

#     instances = [{"content": text} for text in texts]
#     response = model.predict(endpoint=endpoint, instances=instances)
#     summaries = [prediction["content"] for prediction in response.predictions]
#     return summaries

# def generate_embeddings(texts):
#     model = aiplatform.gapic.PredictionServiceClient()
#     endpoint = "projects/adsp-34002-on02-sopho-scribe/locations/us-central1/endpoints/YOUR_EMBEDDING_ENDPOINT_ID"
    
#     instances = [{"content": text} for text in texts]
#     response = model.predict(endpoint=endpoint, instances=instances)
#     embeddings = [prediction["embedding"] for prediction in response.predictions]
#     return embeddings

# def rank_by_relevance(articles, query_embedding):
#     embeddings = generate_embeddings(articles)
#     similarities = cosine_similarity([query_embedding], embeddings)
    
#     ranked_articles = sorted(zip(articles, similarities[0]), key=lambda x: x[1], reverse=True)
#     return ranked_articles

# if __name__ == "__main__":
#     # Step 1: Fetch news
#     articles = fetch_news("Walmart")
    
#     # Step 2: Summarize with Gemini
#     summarized_articles = summarize_with_gemini(articles)
    
#     # Step 3: Rank articles based on relevance to 'Walmart'
#     query_embedding = generate_embeddings(["Walmart"])[0]  # Use the embedding for the query term 'Walmart'
#     ranked_articles = rank_by_relevance(summarized_articles, query_embedding)
    
#     # Output results
#     print("📰 Walmart Market Summary (Ranked):")
#     for idx, (summary, score) in enumerate(ranked_articles, start=1):
#         print(f"\n{idx}. {summary} (Relevance Score: {score:.3f})")